In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Object of this case study :** Given a Bank customer, can we build a classifier which can determine whether they will leave or not?

# Exploratory Data Analysis (EDA)

Draw some plots and find interesting dependencies. 

In [ ]:
# Show the first 5 rows of the data set
# This case study is a classification problem

dataset = pd.read_csv('/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv')
dataset.head()

In [ ]:
# In order to see if there are missing values
pd.isnull(dataset).sum()

In [ ]:
# In order to see the type of values
dataset.info()

In [ ]:
# View the proportion of customer churn

import matplotlib.pyplot as plt
from pylab import rcParams
churnvalue = dataset['Exited'].value_counts()
labels = dataset['Exited'].value_counts().index
rcParams["figure.figsize"] = 6,6
colors = ['#ff9999', '#66b3ff']
plt.pie(churnvalue, labels = labels, colors = colors, explode = (0.1,0),autopct = '%1.1f%%', shadow = True)
plt.title('Proportions of Customer Churn')

In [ ]:
# Divide age ranges
age_bins = [10,50,100]
dataset['age_range'] = pd.cut(x = dataset['Age'], bins = age_bins, labels = ['Non-SeniorCitizen','SeniorCitizen'])
dataset['age_range'].value_counts()

In [ ]:
# Impact of age,gender,HasCrCard and IsActiveMember for the customer churn
import seaborn as sns
fig, axes = plt.subplots(nrows = 2, ncols = 3, figsize = (15,10))

plt.subplot(2,3,1)
gender = sns.countplot(x = 'Gender', hue = 'Exited', data = dataset, palette = 'Set3')
plt.xlabel('gender')
plt.title('Churn by gender')

plt.subplot(2,3,2)
seniorcitizen = sns.countplot(x = 'age_range', hue = 'Exited', data = dataset, palette = 'Set3')
plt.xlabel('age')
plt.title('Churn by age')

plt.subplot(2,3,3)
hascrcard = sns.countplot(x = 'HasCrCard', hue = 'Exited', data = dataset, palette = 'Set3')
plt.xlabel('HasCrCard')
plt.title('Churn by HasCrCard')

plt.subplot(2,3,4)
isactivemember = sns.countplot(x = 'IsActiveMember', hue = 'Exited', data = dataset, palette = 'Set3')
plt.xlabel('IsActiveMember')
plt.title('Churn by IsActiveMember')

plt.subplot(2,3,5)
numofproducts = sns.countplot(x = 'NumOfProducts', hue = 'Exited', data = dataset, palette = 'Set3')
plt.xlabel('NumOfProducts')
plt.title('Churn by NumOfProducts')

plt.subplot(2,3,6)
geography = sns.countplot(x = 'Geography', hue = 'Exited', data = dataset, palette = 'Set3')
plt.xlabel('Geography')
plt.title('Churn by Geography')


In [ ]:
# process the categorical features by using one hot encoder
geography=pd.get_dummies(dataset['Geography'])
gender=pd.get_dummies(dataset['Gender'])

In [ ]:
feature=dataset.drop(['Geography','Gender'],axis=1)
feature=pd.concat([dataset,geography,gender],axis=1)

In [ ]:
label = feature['Exited']
feature = feature.drop(columns = ['RowNumber','CustomerId','Surname','Exited','age_range','Geography','Gender'])

In [ ]:
feature.head()

In [ ]:
# Create observations and labels
X = feature
y = label.values

In [ ]:
#split dataset into train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
# Heatmap
corr = X.corr()
plt.figure(figsize = (10,8))
ax = sns.heatmap(corr, xticklabels = corr.columns, yticklabels = corr.columns,
                linewidths = 0.2, cmap = 'YlGnBu', annot = True)

plt.title("Correlation between variables")

In [ ]:
from sklearn.linear_model import  Lasso
lasso=Lasso(alpha=0.001)
lasso.fit(X_train,y_train)

FI_lasso = pd.DataFrame({"Feature importance":lasso.coef_}, index=X_train.columns)
FI_lasso.sort_values("Feature importance",ascending=False)
FI_lasso[FI_lasso["Feature importance"]!=0].sort_values("Feature importance").plot(kind="barh",figsize=(8,6))
plt.xticks(rotation=90)
plt.show()

# Finetuning different models

In [ ]:
Classify_result = []
names = []

**1. Decision Tree**

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.model_selection import cross_val_score

# Setup the parameters and distributions to sample from: params
params = {"max_features": randint(1, 9),
          "min_samples_leaf": randint(1, 9)}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(param_distributions = params, estimator = tree, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

# Fit it to the data
tree_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {:.2f}".format(tree_cv.best_score_))
print("Test set score : {:.2f}".format(tree_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best estimator {}".format(tree_cv.best_estimator_))

In [ ]:
classifier_tree = DecisionTreeClassifier(max_features=5, min_samples_leaf=7)
tree_score = cross_val_score(classifier_tree,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([tree_score.mean()*100]))
names.append("Decision Tree")
print(tree_score.mean()*100)

**2. Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier  

params = {"n_estimators": randint(1,20),
          "max_features": ['auto', 'sqrt'],
          "min_samples_split" : randint(1, 9),
          "min_samples_leaf": randint(1, 9),
          "max_depth" : [2,4],
          "bootstrap": [True, False]}

RF = RandomForestClassifier()

RF_cv = RandomizedSearchCV(param_distributions = params, estimator = RF, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

RF_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {:.2f}".format(RF_cv.best_score_))
print("Test set score : {:.2f}".format(RF_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(RF_cv.best_params_))
print("Best estimator {}".format(RF_cv.best_estimator_))

In [ ]:
classifier_rf = RandomForestClassifier(bootstrap=False, max_depth=4, max_features='sqrt',
                       min_samples_leaf=4, min_samples_split=7, n_estimators=6) 
rf_score = cross_val_score(classifier_rf,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([rf_score.mean()*100]))
names.append("Random Forest")
print(rf_score.mean()*100)

**3. Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression 
from scipy.stats import loguniform

params = {"solver": ['newton-cg', 'lbfgs', 'liblinear'],
          "C": loguniform(1e-5, 100)}

LR = LogisticRegression()

LR_cv = RandomizedSearchCV(param_distributions = params, estimator = LR, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

LR_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {:.2f}".format(LR_cv.best_score_))
print("Test set score : {:.2f}".format(LR_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(LR_cv.best_params_))
print("Best estimator {}".format(LR_cv.best_estimator_))

In [ ]:
classifier_lr = LogisticRegression(C=25.900095000627836, solver='newton-cg')
lr_score = cross_val_score(classifier_lr,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([lr_score.mean()*100]))
names.append("Logistic Regression")

print(lr_score.mean()*100)

**4. Support Vector Machine**

In [ ]:
from sklearn.svm import SVC  

params = {"C": loguniform(1e0, 1e3),
          "gamma": loguniform(1e-4, 1e-3)} 

svc = SVC()

svc_cv = RandomizedSearchCV(param_distributions = params, estimator = svc, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

svc_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {:.2f}".format(svc_cv.best_score_))
print("Test set score : {:.2f}".format(svc_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(svc_cv.best_params_))
print("Best estimator {}".format(svc_cv.best_estimator_))

In [ ]:
classifier_svc = SVC(C=1.0693196454862999, gamma=0.0003231419346878296)
svc_score = cross_val_score(classifier_svc,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([svc_score.mean()*100]))
names.append("Support Vector Machine")

print(svc_score.mean()*100)

**5. KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier   

params = {"n_neighbors": randint(1, 9),
          "weights": ['uniform', 'distance']} 

knn = KNeighborsClassifier()

knn_cv = RandomizedSearchCV(param_distributions = params, estimator = knn, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

knn_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {:.2f}".format(knn_cv.best_score_))
print("Test set score : {:.2f}".format(knn_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(knn_cv.best_params_))
print("Best estimator {}".format(knn_cv.best_estimator_))


In [ ]:
classifier_knn = KNeighborsClassifier(n_neighbors=6)
knn_score = cross_val_score(classifier_knn,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([knn_score.mean()*100]))
names.append("KNN")

print(knn_score.mean()*100)

**6. Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB 

params = {'var_smoothing': np.logspace(0,-9, num=100)}

GNB = GaussianNB()

GNB_cv = RandomizedSearchCV(param_distributions = params, estimator = GNB, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

GNB_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {:.2f}".format(GNB_cv.best_score_))
print("Test set score : {:.2f}".format(GNB_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(GNB_cv.best_params_))
print("Best estimator {}".format(GNB_cv.best_estimator_))

In [ ]:
classifier_GNB = GaussianNB(var_smoothing=0.3511191734215131)
GNB_score = cross_val_score(classifier_GNB,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([GNB_score.mean()*100]))
names.append("Naive Bayes")

print(GNB_score.mean()*100)

**7.XGBoost**

In [ ]:
from xgboost import XGBClassifier
from scipy import stats
from scipy.stats import randint
    
params = {'n_estimators': randint(1,20),
          'learning_rate': stats.uniform(0.01, 0.6),
          'subsample': stats.uniform(0.3, 0.9),
          'objective': ['binary:logistic'],
          'eval_metric': ['logloss','auc','error'],
          'max_depth': [3, 4, 5, 6, 7, 8, 10, 12, 15],
          'colsample_bytree': stats.uniform(0.5, 0.9),
          'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
          'min_child_weight': [1, 2, 3, 4]}

xgb = XGBClassifier()

xgb_cv = RandomizedSearchCV(param_distributions = params, estimator = xgb, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

xgb_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {}".format(xgb_cv.best_score_))
print("Test set score : {}".format(xgb_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(xgb_cv.best_params_))
print("Best estimator {}".format(xgb_cv.best_estimator_))

In [ ]:
classifier_xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7814542855162242,
              eval_metric='logloss', gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',objective = 'binary:logistic',
              learning_rate=0.39834229466739496, max_delta_step=0, max_depth=3,
              min_child_weight=2, monotone_constraints='()',
              n_estimators=9, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.7174957047484927, tree_method='exact',
              validate_parameters=1, verbosity=None)

xgb_score = cross_val_score(classifier_xgb,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([xgb_score.mean()*100]))
names.append("XGBoost")

print(xgb_score.mean()*100)

**8. CatBoostClassifier**

In [ ]:
from catboost import CatBoostClassifier

params = {'learning_rate': stats.uniform(0.01, 0.6),
          'n_estimators': randint(1,20),
          'max_depth': randint(3, 15)}

cat = CatBoostClassifier()

cat_cv = RandomizedSearchCV(param_distributions = params, estimator = cat, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

cat_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {}".format(cat_cv.best_score_))
print("Test set score : {}".format(cat_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(cat_cv.best_params_))
print("Best estimator {}".format(cat_cv.best_estimator_))

In [ ]:
classifier_cat = CatBoostClassifier(learning_rate = 0.26576502656428785,
                                   max_depth = 14,
                                   n_estimators = 18)

cat_score = cross_val_score(classifier_cat,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([cat_score.mean()*100]))
names.append("CatBoost")

print(cat_score.mean()*100)

**9. AdaBoostClassifier**

In [ ]:
from sklearn.ensemble  import AdaBoostClassifier
ada_classifier = AdaBoostClassifier(random_state=42)

params = {'learning_rate': stats.uniform(0.01, 0.6),
          'n_estimators': randint(1,20)}

ada = AdaBoostClassifier()

ada_cv = RandomizedSearchCV(param_distributions = params, estimator = ada, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

ada_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {}".format(ada_cv.best_score_))
print("Test set score : {}".format(ada_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(ada_cv.best_params_))
print("Best estimator {}".format(ada_cv.best_estimator_))

In [ ]:
classifier_ada = AdaBoostClassifier(learning_rate=0.44268876821307374, n_estimators=16)

ada_score = cross_val_score(classifier_ada,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([ada_score.mean()*100]))
names.append("AdaBoost")

print(ada_score.mean()*100)

**10. GradientBoostingClassifier**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

params = {'learning_rate': stats.uniform(0.01, 0.6),
          'n_estimators': randint(1,20),
          'min_samples_split':[2,4,6],
          'min_samples_leaf':[3,5,7]}


GBC = GradientBoostingClassifier()

GBC_cv = RandomizedSearchCV(param_distributions = params, estimator = GBC, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

GBC_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {}".format(GBC_cv.best_score_))
print("Test set score : {}".format(GBC_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(GBC_cv.best_params_))
print("Best estimator {}".format(GBC_cv.best_estimator_))

In [ ]:
classifier_GBC = GradientBoostingClassifier(learning_rate=0.41366238767540275,
                           min_samples_leaf=7, min_samples_split=6,
                           n_estimators=11)

GBC_score = cross_val_score(classifier_GBC,X_train,y_train,cv=5)

Classify_result.append(pd.DataFrame([GBC_score.mean()*100]))
names.append("Gradient Boosting")

print(GBC_score.mean()*100)

# Result

In [ ]:
names=pd.DataFrame(names)
names=names[0].tolist()
result=pd.concat(Classify_result,axis=1)
result.columns=names
result.index=["accuracy"]
result

**Comment :** In this case, 10 classifiers havs been tested, The best performante models are the following three : Gradient Boosting, XGBoost and CatBoost. 

**1. Question :** Why you chose your solution you arrived at you solution?

We used the "RandomizedResearchCV" method, to be precise, we applied the "RandomizedResearch" method to fine tune the parameters, and used the cross-validation method to avoid the contingency of the results. Once we obtained the best model, we initiated the model with these parameters and calculate the score of accuracy using cross validation. And the tree-ensemble-based method always performs better from the resulted we got.

**2. Question :** If you are satisfied with your solution and what else you would do to make it better?

In order to make the model better, we can focus on feature enineering. 
- In these experiments above, we used all features for modeling, but in fact we can discard the features with the lowest contribution one by one to improve the model performance until the model performance converges. 
- We found that there are some features whose data have large differences, for example, the columnes Balance, EstimatedSalary and Tenure, so we need to standardize them.


In [ ]:
# Standardize the data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=False)
feature[['Tenure','Balance','EstimatedSalary']] = scaler.fit_transform(feature[['Tenure','Balance','EstimatedSalary']])

In [ ]:
feature.head()

In [ ]:
X = feature
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
params = {'n_estimators': randint(1,20),
          'learning_rate': stats.uniform(0.01, 0.6),
          'subsample': stats.uniform(0.3, 0.9),
          'objective': ['binary:logistic'],
          'eval_metric': ['logloss','auc','error'],
          'max_depth': [3, 4, 5, 6, 7, 8, 10, 12, 15],
          'colsample_bytree': stats.uniform(0.5, 0.9),
          'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
          'min_child_weight': [1, 2, 3, 4]}

xgb = XGBClassifier()

xgb_cv = RandomizedSearchCV(param_distributions = params, estimator = xgb, scoring = "accuracy", n_iter = 5, cv = 5, verbose = 1)

xgb_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Best score on train set is {}".format(xgb_cv.best_score_))
print("Test set score : {}".format(xgb_cv.score(X_test, y_test)))
print("Tuned Decision Tree Parameters: {}".format(xgb_cv.best_params_))
print("Best estimator {}".format(xgb_cv.best_estimator_))

In [ ]:
classifier_xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6901197675703821,
              eval_metric='error', gamma=0.1, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.45970426128318287, 
              max_delta_step=0, max_depth=3, min_child_weight=4, objective = 'binary:logistic',
              monotone_constraints='()', n_estimators=17, n_jobs=4,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.9826131771966484,
              tree_method='exact', validate_parameters=1, verbosity=None)

xgb_score = cross_val_score(classifier_xgb,X_train,y_train,cv=5)
print(xgb_score.mean()*100)